# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [4]:
import pandas as pd
import numpy as np
import xlsxwriter
import math
import requests

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [5]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [6]:
from secrets import IEX_CLOUD_API_TOKEN
IEX_CLOUD_API_TOKEN

'Tsk_a4d1a9845c1d4bd68caeebb6ad30ec3f'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [7]:
ticker = 'AAPL'
url = f'https://sandbox.iexapis.com/stable/stock/{ticker}/quote/?filter=symbol,latestPrice,marketCap&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(url)
symbol = data.json()['symbol']
latestPrice = data.json()['latestPrice']
marketCap = data.json()['marketCap']
print(symbol)
print(latestPrice)
print(marketCap)

AAPL
133.66
2276771518150


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [8]:
#Calling one stock at a time

my_columns = ['Ticker','LatestPrice','MarketCap','NumSharesToBuy']
stocks_dataframe = pd.DataFrame(columns=my_columns)

stocks_list = stocks['Ticker'].values.tolist()
for stock in stocks_list:
    url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?filter=symbol,latestPrice,marketCap&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(url)
    symbol = data.json()['symbol']
    latestPrice = data.json()['latestPrice']
    marketCap = data.json()['marketCap']
    stocks_dataframe = stocks_dataframe.append(
            pd.Series(
            [
                symbol,
                latestPrice,
                marketCap,
                'N/A'
            ],
                index=my_columns),
            ignore_index=True
            )
stocks_dataframe
    

,Ticker,LatestPrice,MarketCap,NumSharesToBuy
0,A,121.48,37261170576,N/A
1,AAL,15.88,9668605922,N/A
2,AAP,164.32,10892470394,N/A
3,AAPL,133.87,2280310584396,N/A
4,ABBV,108.75,195567687040,N/A
...,...,...,...,...
500,YUM,112.07,32989538095,N/A
501,ZBH,161.15,32433598052,N/A
502,ZBRA,400.73,21019083376,N/A
503,ZION,43.91,7393342756,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [9]:
#Batch Stock API Call

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
        
my_columns = ['Ticker','LatestPrice','MarketCap','NumSharesToBuy']

stocks_dataframe = pd.DataFrame(columns=my_columns)

stocks_list = stocks['Ticker'].values.tolist()

stocks_group_list = list(chunks(stocks_list,100))
for stock_group in stocks_group_list:
    stock_group = str(stock_group).replace("'","").replace('[','').replace(']','').replace(' ','')
    batch_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={stock_group}&types=quote&filter=symbol,latestPrice,marketCap&token={IEX_CLOUD_API_TOKEN}'
    stock_batch_data = requests.get(batch_url).json()
    for symbol in stock_batch_data:
        latest_price = stock_batch_data[symbol]['quote']['latestPrice']
        market_cap = stock_batch_data[symbol]['quote']['marketCap']
        stocks_dataframe = stocks_dataframe.append(
            pd.Series(
            [
                symbol,
                latest_price,
                market_cap,
                'N/A'
            ],
                index=my_columns),
            ignore_index=True
            )
stocks_dataframe
    

,Ticker,LatestPrice,MarketCap,NumSharesToBuy
0,A,119.66,38050347600,N/A
1,AAL,16.26,9572524298,N/A
2,AAP,160.10,10963213394,N/A
3,AAPL,138.32,2367683352720,N/A
4,ABBV,109.58,197216348593,N/A
...,...,...,...,...
500,YUM,111.26,33078560024,N/A
501,ZBH,159.66,32455133094,N/A
502,ZBRA,396.44,20946800093,N/A
503,ZION,45.30,7384545797,N/A


In [10]:
#Ask User for Portfolio Amount
validInput = False
while validInput == False:
    try:
        portfolioAmount = float(input('What is your portfolio amount? '))
        validInput = True
    except:
        print('Please enter a number')
print(portfolioAmount)
positionSize = portfolioAmount/len(stocks_dataframe.index)
print(positionSize)

What is your portfolio amount? 800000
800000.0
1584.1584158415842


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [11]:
for i in range(len(stocks_dataframe.index)):
    stocks_dataframe.loc[i,'NumSharesToBuy'] = math.floor(positionSize/stocks_dataframe.loc[i,'LatestPrice'])
stocks_dataframe

,Ticker,LatestPrice,MarketCap,NumSharesToBuy
0,A,119.66,38050347600,13
1,AAL,16.26,9572524298,97
2,AAP,160.10,10963213394,9
3,AAPL,138.32,2367683352720,11
4,ABBV,109.58,197216348593,14
...,...,...,...,...
500,YUM,111.26,33078560024,14
501,ZBH,159.66,32455133094,9
502,ZBRA,396.44,20946800093,3
503,ZION,45.30,7384545797,34


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [58]:
writer = pd.ExcelWriter('recommended_trades.xlsx',engine='xlsxwriter')
stocks_dataframe.to_excel(writer,'Recommended_Trades',index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [59]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(

    {
    'font_color':font_color,
    'bg_color':background_color,
    'border':1
    }
)

dollar_format = writer.book.add_format(

    {
    'num_format': '$0.00',
    'font_color':font_color,
    'bg_color':background_color,
    'border':1
    }
)

integer_format = writer.book.add_format(

    {
    'num_format':'0',
    'font_color':font_color,
    'bg_color':background_color,
    'border':1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [60]:
# writer.sheets['Recommended_Trades'].set_column('A:A',18,string_format)
# writer.sheets['Recommended_Trades'].set_column('B:B',18,dollar_format)
# writer.sheets['Recommended_Trades'].set_column('C:C',18,dollar_format)
# writer.sheets['Recommended_Trades'].set_column('D:D',18,integer_format)

# writer.sheets['Recommended_Trades'].write('A1','Ticker',string_format)
# writer.sheets['Recommended_Trades'].write('B1','Stock Price',string_format)
# writer.sheets['Recommended_Trades'].write('C1','Market Cap',string_format)
# writer.sheets['Recommended_Trades'].write('D1','Num Shares To Buy',string_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [61]:
column_formats = {
    'A':['Ticker',string_format],
    'B':['Stock Price',dollar_format],
    'C':['Market Cap',dollar_format],
    'D':['Num Shares To Buy',integer_format]
}

for column_letter in column_formats.keys(): # first line is column header and second line is column data.
    writer.sheets['Recommended_Trades'].write(f'{column_letter}1',column_formats[column_letter][0],string_format)
    writer.sheets['Recommended_Trades'].set_column(f'{column_letter}:{column_letter}',18,column_formats[column_letter][1])

## Saving Our Excel Output

Saving our Excel file is very easy:

In [62]:
writer.save()